In [ ]:
from smolagents import CodeAgent, FinalAnswerTool, load_tool, tool
from smolagents.models import InferenceClientModel
from huggingface_hub import InferenceClient
from Gradio_UI import GradioUI
import datetime
import pytz
import yaml
import requests
import config_keys

# ---- TOOLS ----
@tool
def final_answer(answer: str) -> str:
    """
    Use this to return the final answer to the user and stop execution.
    Args:
        answer: The final response to send back to the user.
    """
    return answer

@tool
def my_custom_tool(arg1: str, arg2: int) -> str:
    """A tool that does nothing yet.
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        tz = pytz.timezone(timezone)
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

@tool
def get_weather_weatherapi(city: str) -> str:
    """
    Fetches the current weather in a given city using WeatherAPI.com.

    Args:
        city: The name of the city to get weather for.

    Returns:
        A plain-text weather report including temperature, condition, humidity, and wind speed.
    """
    api_key = config_keys.weather_api_key
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {"key": api_key, "q": city, "aqi": "no"}

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        loc = data["location"]
        current = data["current"]
        return (
            f"Weather Report:\n"
            f"Location: {loc['name']}, {loc['country']}\n"
            f"Temperature: {current['temp_c']}°C\n"
            f"Condition: {current['condition']['text']}\n"
            f"Humidity: {current['humidity']}%\n"
            f"Wind: {current['wind_kph']} km/h"
        )
    return f"Error fetching weather: {response.status_code} - {response.text}"

In [ ]:
# ---- Finance Tools ----
@tool
def get_stock_info_polygon(ticker: str) -> str:
    """
    Fetches the latest stock data for a given ticker symbol using the Polygon.io API.

    Args:
        ticker: The stock ticker symbol (e.g., "AAPL", "MSFT").

    Returns:
        A plain-text summary of the stock’s current price and change.
    """
    api_key = "YOUR_POLYGON_API_KEY"  # Replace with your actual API key
    url = f"https://api.polygon.io/v2/last/trade/{ticker.upper()}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        trade = data.get("results", {})
        price = trade.get("p")
        exchange = trade.get("x")
        timestamp = trade.get("t")
        if price is not None:
            return (
                f"Stock Info:\n"
                f"Ticker: {ticker.upper()}\n"
                f"Last Trade Price: ${price:.2f}\n"
                f"Exchange: {exchange}\n"
                f"Timestamp: {timestamp}"
            )
        else:
            return f"No trade data available for ticker '{ticker.upper()}'."
    return f"Error fetching stock data: {response.status_code} - {response.text}"

In [ ]:
# ---- MODEL ----
client = InferenceClient(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    token=config_keys.hugging_face_api_key
)
model = InferenceClientModel(
    client=client,
    model_id="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature=0.5,
    max_tokens=2048
)

In [ ]:
# ---- TOOLS + PROMPTS ----
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)

In [ ]:
# ---- AGENT ----
agent = CodeAgent(
    model=model,
    tools=[
        final_answer,
        my_custom_tool,
        get_current_time_in_timezone,
        get_weather_weatherapi,
        image_generation_tool,
    ],
    prompt_templates=prompt_templates,
    max_steps=6,
    verbosity_level=1,
    grammar=None,
    planning_interval=None,
    additional_authorized_imports=["requests", "pytz", "datetime"]
)


# ---- LAUNCH UI ----
GradioUI(agent).launch()